In [1]:
import pandas as pd
import ast
from datetime import datetime

dtype_dict = {
    'deal_id': str,
    'pb_id': str,
    'name': str,
    'deal_number': int,
    'year_founded': 'Int64',
    'deal_size': float,
    'post_valuation': float,
    'total_vc_funding': float,
    'total_invested_equity': float,
    'deal_type': str,
    'deal_type2': str,
    'vc_round': str,
    'lead_investor': str,
    'company_website': str,
    'hq_location': str,
    'description': str,
    'primary_industry_code': str,
    'primary_industry_group': str,
    'primary_industry_sector': str
}

def parse_list(x):
    if pd.isna(x) or x == '[]':
        return []
    try:
        return ast.literal_eval(x)
    except:
        return []

def parse_date(x):
    if pd.isna(x):
        return None
    try:
        return datetime.strptime(x, '%Y-%m-%d')
    except:
        return None

def read_data(path: str):

    df = pd.read_csv(path, 
                     sep='\t',
                     dtype=dtype_dict,
                     converters={
                         'deal_date': parse_date,
                         'investors': parse_list,
                         'investor_ids': parse_list,
                         'first_time_investors': parse_list,
                         'keywords': parse_list,
                         'verticals': parse_list
                     },
                     na_values=['', 'nan', 'NULL'])
    
    df['post_valuation_status'] = df['post_valuation_status'].astype('category')
    return df

def convert_vc_round(round_str):
    if pd.isna(round_str) or not round_str:
        return None
    
    mapping = {
        '1st': 1,
        '2nd': 2,
        '3rd': 3,
        '4th': 4,
        '5th': 5,
        '6th': 6,
        '7th': 7,
        '8th': 8,
        '9th': 9,
        '10th': 10,
        'Angel': 0,
    }
    
    round_lower = round_str.lower()
    for key in mapping:
        if key in round_lower:
            return mapping[key]
    return None

In [2]:
df = read_data(path="./fundraising.tsv")

In [3]:
def get_early_stage_deals(df: pd.DataFrame, threshold_millions: int) -> pd.DataFrame:
    # Sort by date and calculate cumulative funding
    df_sorted = df.sort_values(['pb_id', 'deal_number'])
    df_sorted['cumulative_funding'] = df_sorted.groupby('pb_id')['deal_size'].cumsum()
    
    # Get first deal that takes the firm over the threshold
    result = (df_sorted[df_sorted['cumulative_funding'] >= threshold_millions]
              .groupby('pb_id')
              .first()
              .reset_index())
    
    return result

In [4]:
def has_complete_deals(deals):
        expected_range = range(1, max(deals) + 1)
        return set(deals) == set(expected_range)

def remove_incomplete_firms(df: pd.DataFrame) -> pd.DataFrame:
    complete_firms = df.groupby('pb_id')['deal_number'].agg(has_complete_deals)
    return df[df['pb_id'].isin(complete_firms[complete_firms].index)]

# remove firms with missing deals
# remove Accellerator deals (optionally add an indicator var to the firm)
# remove firms with deals that have missing values
# subset to early stage deals

def add_post_accelerator(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # Find accelerator deals for each firm
    accelerator_dates = df[
        (df['deal_type'] == 'Accelerator/Incubator')
    ].groupby('pb_id')['deal_date'].min()
    
    # Mark deals that came after accelerator
    df['post_accelerator'] = False
    for pb_id, acc_date in accelerator_dates.items():
        df.loc[
            (df['pb_id'] == pb_id) & 
            (df['deal_date'] > acc_date), 
            'post_accelerator'
        ] = True
    
    return df

accelerator_ind_df = add_post_accelerator(df)

In [7]:
print(f"len(df)={len(df)}")

print(f"len(accelerator_ind_df)={len(accelerator_ind_df)}")
complete_firms_df = remove_incomplete_firms(accelerator_ind_df)

print(f"len(complete_firms_df)={len(complete_firms_df)}")
non_accellerator_deals_df = accelerator_ind_df[~accelerator_ind_df['deal_id'].isin(
    accelerator_ind_df[accelerator_ind_df.deal_type == 'Accelerator/Incubator']['deal_id']
)]

print(f"len(non_accellerator_deals_df)={len(non_accellerator_deals_df)}")
firms_with_known_deal_sizes_df = non_accellerator_deals_df[non_accellerator_deals_df['deal_size'].notna()]

print(f"len(firms_with_known_deal_sizes_df)={len(firms_with_known_deal_sizes_df)}")


len(df)=76546
len(accelerator_ind_df)=76546
len(complete_firms_df)=35218
len(non_accellerator_deals_df)=69191
len(firms_with_known_deal_sizes_df)=64321


In [8]:
early_stage_deals = get_early_stage_deals(firms_with_known_deal_sizes_df, 10)
print(f"len(early_stage_deals)={len(early_stage_deals)}")
early_stage_deals

len(early_stage_deals)=15917


,pb_id,deal_id,name,deal_number,year_founded,deal_date,deal_size,post_valuation_status,post_valuation,total_vc_funding,...,company_website,hq_location,description,primary_industry_code,primary_industry_group,primary_industry_sector,keywords,verticals,post_accelerator,cumulative_funding
0,100022-14,175697-20T,Spinn Coffee,6,2014,2022-01-10,34.74,Actual,133.74,44.47,...,www.spinn.com,"San Francisco, CA",Developer of internet-connected coffee machine...,Household Appliances,Consumer Durables,Consumer Products and Services (B2C),"[automated coffee making, coffee maker, coffee...","[FoodTech, Internet of Things, TMT]",True,44.02
1,100024-12,86665-96T,Maven,2,2014,2017-11-20,11.00,Actual,36.18,15.47,...,www.mavenclinic.com,"New York, NY",Developer of a digital health platform designe...,Clinics/Outpatient Services,Healthcare Services,Healthcare,"[care coordination, care navigation, digital c...","[FemTech, HealthTech, Mobile, SaaS]",False,15.47
2,100099-09,70049-98T,Weaveworks,2,2014,2016-05-11,15.00,Actual,70.00,20.00,...,www.weave.works,"San Francisco, CA",Developer of a software as a service-based sof...,Software Development Applications,Software,Information Technology,"[cloud computing saas, cloudtech and devops, c...","[CloudTech & DevOps, SaaS]",False,20.00
3,100100-26,95647-24T,Wonder (Artificial Intelligence & Machine Lear...,2,2012,2017-06-30,12.60,Actual,82.00,14.60,...,www.askwonder.com,"Dallas, TX",Developer of a desk research platform designed...,Business/Productivity Software,Software,Information Technology,"[business research services, content services,...","[Artificial Intelligence & Machine Learning, S...",False,14.60
4,100117-36,69514-66T,Shape Memory Medical,6,2009,2017-07-19,14.00,Actual,36.00,15.62,...,www.shapemem.com,"Santa Clara, CA",Developer of a medical device designed to cate...,Therapeutic Devices,Healthcare Devices and Supplies,Healthcare,"[cardiovascular, cardiovascular system, memory...","[HealthTech, Life Sciences]",True,15.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15912,99939-25,39874-33T,KnuEdge,1,2005,2005-01-28,12.00,NaN,NaN,12.00,...,www.knuedge.com,"San Diego, CA",Developer of next-generation voice biometric t...,Business/Productivity Software,Software,Information Technology,"[computational system, natural language techno...","[Artificial Intelligence & Machine Learning, TMT]",False,12.00
15913,99943-21,62715-97T,Still Austin Whiskey,2,2014,2018-12-18,15.40,Actual,36.40,16.48,...,www.stillaustin.com,"Austin, TX",Producer of a craft whiskey distillery intende...,Beverages,Consumer Non-Durables,Consumer Products and Services (B2C),"[alcohol wine, drinks cans, food cafe, gin dis...",[],False,16.48
15914,99995-68,75683-35T,Austin Eastciders,4,2013,2017-10-12,10.42,Actual,29.42,16.75,...,www.austineastciders.com,"Austin, TX",Producer of traditionally crafted ciders inten...,Beverages,Consumer Non-Durables,Consumer Products and Services (B2C),"[cider company, cider maker, cider producer, c...",[E-Commerce],False,16.75
15915,99995-95,95638-24T,AlertMedia,4,2013,2017-10-11,8.57,Actual,42.87,17.58,...,www.alertmedia.com,"Austin, TX",Developer of a cloud-based emergency mass comm...,Communication Software,Software,Information Technology,"[alerts monitoring, emergency communication, e...","[Climate Tech, SaaS, TMT]",False,17.58


In [9]:
from collections import Counter

def count_firms_per_tag(df: pd.DataFrame, tag_column: str) -> Counter:
    tag_counts = Counter()
    for pb_id, tags in df.groupby('pb_id')[tag_column].first().items():
        for tag in tags:
            tag_counts[tag] += 1
    return tag_counts

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import pandas as pd
from typing import Dict, Tuple, Any
from scipy import stats

def prediction_interval(random_forest, X_pred, percentile=95):
    """Calculate prediction intervals using out-of-bag estimates"""
    predictions = []
    for estimator in random_forest.estimators_:
        predictions.append(estimator.predict(X_pred))
    predictions = np.array(predictions)
    lower = np.percentile(predictions, (100 - percentile) / 2, axis=0)
    upper = np.percentile(predictions, 100 - (100 - percentile) / 2, axis=0)
    return lower, upper

def predict_deal_size(df: pd.DataFrame, n_folds = 5) -> tuple:
    # Prepare categorical features
    le_location = LabelEncoder()
    le_industry = LabelEncoder()
    mlb_keywords = MultiLabelBinarizer()
    mlb_verticals = MultiLabelBinarizer()
    
    # Transform features
    X_location = le_location.fit_transform(df['hq_location']).reshape(-1, 1)
    X_industry = le_industry.fit_transform(df['primary_industry_code']).reshape(-1, 1)
    X_keywords = mlb_keywords.fit_transform(df['keywords'].tolist())
    X_verticals = mlb_verticals.fit_transform(df['verticals'].tolist())
    
    # Combine all features
    X = np.hstack([
        X_location,
        X_industry,
        X_keywords,
        X_verticals,
        df[['post_accelerator', 'cumulative_funding']].values
    ])
    
    y = df['deal_size'].values
    
    # Initialize model
    rf = RandomForestRegressor(n_estimators=10, random_state=42)
    
    # Perform k-fold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    cv_scores = {
        'r2': cross_val_score(rf, X, y, cv=kf, scoring='r2'),
        'neg_rmse': cross_val_score(rf, X, y, cv=kf, scoring='neg_root_mean_squared_error'),
        'neg_mae': cross_val_score(rf, X, y, cv=kf, scoring='neg_mean_absolute_error')
    }

    # Train the model on all the data, now that we've cross-validated
    rf.fit(X, y)
    
    # Make predictions
    y_pred = rf.predict(X)
    residuals = y - y_pred

    # Calculate intervals for training data
    lower_bounds, upper_bounds = prediction_interval(rf, X)

    # Check distribution of residuals, hopefully random
    normality_test = stats.normaltest(residuals)
    
    # Get feature importance
    feature_names = (
        ['location', 'industry'] +
        mlb_keywords.classes_.tolist() +
        mlb_verticals.classes_.tolist() +
        ['post_accelerator', 'cumulative_funding']
    )
    
    importance_dict = dict(zip(feature_names, rf.feature_importances_))

    results = {
        'model': rf,
        'feature_importance': importance_dict,
        'cross_validation': {
            'r2_scores': cv_scores['r2'],
            'r2_mean': cv_scores['r2'].mean(),
            'r2_std': cv_scores['r2'].std(),
            'rmse_scores': -cv_scores['neg_rmse'],
            'rmse_mean': -cv_scores['neg_rmse'].mean(),
            'rmse_std': -cv_scores['neg_rmse'].std(),
            'mae_scores': -cv_scores['neg_mae'],
            'mae_mean': -cv_scores['neg_mae'].mean(),
            'mae_std': -cv_scores['neg_mae'].std()
        },
        'residual_analysis': {
            'residuals': residuals,
            'normality_test_statistic': normality_test.statistic,
            'normality_test_p_value': normality_test.pvalue
        },
        'prediction_intervals': {
            'lower_bounds': lower_bounds,
            'upper_bounds': upper_bounds
        },
        'encoders': {
            'location': le_location,
            'industry': le_industry,
            'keywords': mlb_keywords,
            'verticals': mlb_verticals
        }
    }
    
    return results

In [11]:
%%time
results = predict_deal_size(early_stage_deals)

CPU times: user 41min 22s, sys: 39.9 s, total: 42min 2s
Wall time: 50min 15s


In [12]:
results

{'model': RandomForestRegressor(n_estimators=10, random_state=42),
 'feature_importance': {'location': np.float64(0.00046934169276340823),
  'industry': np.float64(6.201047757834213e-08),
  '1st party data': np.float64(0.0),
  '3 d printing service': np.float64(0.0),
  '3-d technology': np.float64(0.0),
  '32 kwh lithium-ion battery': np.float64(2.0428386588521675e-08),
  '340b management software': np.float64(5.4011425317225075e-08),
  '340b software': np.float64(0.0),
  '340b solutions': np.float64(0.0),
  '360 cameras technology': np.float64(0.0),
  '360 degree video': np.float64(0.0),
  '360 imaging': np.float64(0.0),
  '360 video': np.float64(5.946538420931858e-13),
  '3d animation': np.float64(0.0),
  '3d audio technology': np.float64(4.354984855177056e-08),
  '3d avatar creation': np.float64(3.94659914663562e-10),
  '3d bioprinting': np.float64(0.0),
  '3d bioprinting technology': np.float64(0.0),
  '3d capture': np.float64(0.0),
  '3d capture systems': np.float64(0.0),
  '3d ca

In [21]:
[t.tree_.max_depth for t in results['model'].estimators_]

[103, 120, 82, 81, 102, 92, 96, 127, 89, 118]

In [25]:
importance_dict = results['feature_importance']
{k: importance_dict[k] for k in sorted(importance_dict.keys(), key=lambda k: -1 * importance_dict[k])[0:50]}

{'cumulative_funding': np.float64(0.9689046514710977),
 'automated transportation': np.float64(0.02385573340452695),
 'human capital management': np.float64(0.002139400842357034),
 'location': np.float64(0.00046934169276340823),
 'global trading': np.float64(0.00022547813351037228),
 'space transportation technology': np.float64(0.00017920418600157574),
 'gaming studio': np.float64(0.00017372013267444192),
 'erp': np.float64(0.00014817361810537779),
 'Artificial Intelligence & Machine Learning': np.float64(8.691809592440341e-05),
 'online gaming': np.float64(8.164006029571288e-05),
 'digital game developer': np.float64(7.83494290152556e-05),
 'TMT': np.float64(6.075213195490382e-05),
 'post_accelerator': np.float64(5.9888540907561776e-05),
 'blockchain platform': np.float64(5.679485839598737e-05),
 'Life Sciences': np.float64(5.151623506980355e-05),
 'SaaS': np.float64(5.089144046080033e-05),
 'global energy': np.float64(4.9225895710203474e-05),
 'scm': np.float64(3.545319100359307e-05

In [239]:
len(early_stage_deals)

16415